# Обучение с использованием модели и внутренней мотивации на основе ошибки модели

В данном семинаре рассмотрим вариации модельного обучения на примере табличных алгоритмов. План будет следующий. Вначале мы оттолкнемся от классического алгоритма Dyna-Q — модификации Q-обучения, использующей идеальную табличную модель среды. Данный алгоритм работает с детерминированными средами, а значит обучение модели равносильно запоминанию переходов (т.к. каждый переход можно выучить за один шаг обучения).

После реализации Dyna-Q представим, что модель среды обучается не моментально и может содержать ошибки. Моделируя среду таким образом, мы покроем одновременно два случая: стохастическая среда (распределения переходов придется учить даже в табличном случае) и нейросетевая апроксимация модели (тоже требует постепенного обучения на множестве примеров переходов). Посмотрим для этого варианта, насколько сильно неидеальность обучаемой модели нивелирует преимущества модельного обучения.

Далее посмотрим, как можно построить сигнал внутренней мотивации на основе обучаемой модели и использовать его для ускорения обучения модели и исследования агентом среды. Также попробуем вариацию стратегии генерации воображаемых переходов для обучения стратегии — будем сэмплировать из модели не отдельные случайные переходы (как это сделано в Dyna-Q), а целые воображаемые траектории по аналогии с методами типа Dreamer. Причем эти траектории будут стартовать с текущего состояния агента, имитируя локальное планирование поведения с целью локального доуточнения Q-функции и стратегии по аналогии с методами на базе MCTS (Monte-Carlo Tree Search).

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from collections import deque
from IPython.display import clear_output

%matplotlib inline

In [ ]:
def show_progress(avg_returns):
    """
    Удобная функция, которая отображает прогресс обучения.
    """
    clear_output(True)
    plt.figure(figsize=[12, 4])
    plt.subplot(1, 1, 1)
    plt.plot(*zip(*avg_returns), label='Mean return')
    plt.legend(loc=4)
    plt.grid()
    plt.show()

def compare_logs(logs):
    """Функция сравнения кривых обучения"""
    plt.figure(figsize=[12, 6])
    for log, method_name in logs:
        plt.plot(*zip(*log), label=f'{method_name}')
        plt.legend()
    plt.grid()
    plt.show()

In [ ]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
env.reset(seed=13)
plt.imshow(env.render())

Для начала реализуем класс `Model` обучаемой идеальной модели мира будущего Dyna-Q агента. Некоторые аспекты модели сейчас могут показаться излишними — они нам пригодятся позднее.

(3 балла)

In [ ]:
class Model:
    def __init__(self, n_states, n_actions, seed):
        # проинициализируйте атрибуты, необходимые вам для хранения
        # модели (переходов и вознаграждения)
        # например, можно хранить [в виде таблицы] табличные функции
        #        r: (s, a) -> r
        #   next_s: (s, a) -> next_s
        # нужно так же дополнительно держать таблицы масок
        # для посещенных состояний s `mask_state`
        # и пар (s, a) `mask_state_action`
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        self._rng = np.random.default_rng(seed)

    def add(self, s: int, a: int, r: float, next_s: int) -> float:
        # реализуйте мгновенное обучение модели
        # для (s, a) -> r и (s, a) -> next_s
        # не забудьте пометить посещения в масках
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        # вернем также вознаграждение (понадобится позже)
        return r

    def sample_state(self) -> int:
        # просэмплируйте и верните одно посещенное состояние
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        return s

    def sample_action(self, s) -> int:
        # просэмплируйте и верните одно совершенное ранее действие
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        return a

    def predict_transition(self, s, a) -> tuple[float, int, float]:
        # реализуйте [идеальное] предсказание моделей
        # перехода и вознаграждения
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        # дополнительно вернем степень уверенности в предсказании
        # в текущем случае она равна значению в маске (=1)
        confidence = self.mask_state_action[s, a]
        return self.r[s, a], next_s, confidence

    def sample(self) -> tuple[int, int, float, int, float]:
        # реализуйте выбор одного случайного перехода
        # и верните его вместе с уверенностью в предсказании
        # (s, a, r, next_s, confidence)
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################

Теперь создадим класс `DynaQAgent`, который будет решать задачу обучения методом Dyna-Q, согласно приведенному на рисунке алгоритму:

![dyna](https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/10/dyna.png)

Добавим в реализации возможность выбора исследовательской стратегии между $\epsilon$-жадной и softmax-стратегией. Выбор будет осуществляться на основе передаваемых в конструктор параметров: если передан `eps`, значит первый вариант, если `temp` (температура softmax), то второй.

(2 балла)

In [ ]:
def softmax(x, temp: float = 1.0):
    e_x = np.exp((x - np.max(x)) / temp)
    return e_x / np.sum(e_x)

class DynaQAgent:
    def __init__(
        self, n_states, n_actions, lr, gamma, f_model, seed,
        *, eps=None, temp=None
    ):
        self.Q = np.zeros((n_states, n_actions))
        self.model = f_model(n_states, n_actions, seed=seed)
        self.lr = lr
        self.gamma = gamma
        self.eps = eps
        self.temp = temp
        self.n_actions = n_actions
        self._rng = np.random.default_rng(seed)

    def act(self, s, greedy=False):
        # выбираем действие, используя eps-greedy или softmax
        # исследование среды
        softmax_expl = self.temp is not None

        if softmax_expl and not greedy:
            action = self._rng.choice(
                self.n_actions, p=softmax(self.Q[s], temp=self.temp)
            )
        elif (
            not softmax_expl and not greedy
            and self._rng.random() < self.eps
        ):
            action = self._rng.choice(self.n_actions)
        else:
            action = np.argmax(self.Q[s])  # используем Q-функцию
        return action

    def update(self, s, a, r, s_n, terminated, update_model: bool):
        if update_model:
            r = self.update_model(s, a, r, s_n)

        # реализуйте шаг Q-обучения
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################

    def update_model(self, s, a, r, s_n):
        return self.model.add(s, a, r, s_n)

    def dream(self, max_steps, **_):
        # Совершает шаги обучения на `max_steps`
        # сгенрированных моделью переходах
        for _ in range(max_steps):
            # Добавьте шаг обучения с использованием модели на одном
            # сгенерированном моделью переходе
            # NB: будем использовать значение confidence от модели
            # как вероятность использовать сгенерированный переход
            # для шага обучения
            # NB2: флаг `terminated` будем передавать False
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################

In [ ]:
def train_episode(rng, env, agent, on_model_updates, is_eval=False):
    """Провести один эпизод обучения или тестирования."""
    state, _ = env.reset(seed=int(rng.integers(1_000_000)))
    episode_return = 0.0
    while True:
        action = agent.act(state, greedy=is_eval)

        # выполняем действие в среде
        next_state, reward, terminated, truncated, info = env.step(action)
        if not is_eval:
            # шаг обучения
            agent.update(
                state, action, reward, next_state, terminated,
                update_model=True
            )
        state = next_state
        episode_return += reward
        if terminated or truncated:
            break
        if not is_eval:
            # шаги обучения по модельным данным
            agent.dream(on_model_updates, state=state)
    return episode_return

def train(
    env, agent, n_episodes, on_model_updates, seed,
    show_progress_schedule=100, show_model_progress=False
):
    train_progress = []
    returns_batch = deque(maxlen=4*show_progress_schedule)
    rng = np.random.default_rng(seed)

    for i in range(1, n_episodes+1):
        # один эпизод обучения
        train_episode(rng, env, agent, on_model_updates)
        # и один эпизод тестирования
        returns_batch.append(
            train_episode(rng, env, agent, on_model_updates, is_eval=True)
        )

        if i % show_progress_schedule == 0 and not show_model_progress:
            # график прогресса: отдача за эпизод
            train_progress.append(
                (i, np.mean(returns_batch))
            )
            show_progress(train_progress)
            print(
                f"Episode: {i}, Return: {returns_batch[-1]}, "
                f"AvgReturn[{show_progress_schedule}]: {train_progress[-1][1]:.0f}"
            )
        if i % show_progress_schedule == 0 and show_model_progress:
            # график прогресса: доля выученности посещенных состояний среды
            visited_mask = agent.model.mask_state_action > 0
            avg_model_knowledge = np.mean(
                agent.model.mask_state_action[visited_mask]
            )
            train_progress.append(
                (i, avg_model_knowledge)
            )
            show_progress(train_progress)

    return train_progress

### Q-learning

Обратите внимание, параметр `on_model_updates`, отвечающий на каждом шаге в среде за число дополнительно сгенерированного моделью опыта, равен 0 $\Rightarrow$ значит мы запускаем обычный Q-learning и модель хоть и учим, но ей не пользуемся

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=Model
)

log_q = train(env, agent, n_episodes=4000, on_model_updates=0, seed=seed)

(1 балл)

### DynaQ

1. Сравните скорость обучения алгоритмов Q-обучение и Dyna-Q с параметром `on_model_updates` равным 10.
2. Сравните скорость обучения Dyna-Q при различных `on_model_updates`. В каком случае получились лучшие результаты?

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=Model
)
log_dyna_q = train(env, agent, n_episodes=4000, on_model_updates=5, seed=seed)

## Dyna-Q с неидеальной моделью мира

Давайте теперь представим, что мы ничего не знаем о среде. В частности, мы будем даже не уверены в том, что она детерминированная. Будем именно "учить" переходы с некоторой скоростью обучения (параметром `lr` $\in (0, 1)$). Сделаем это следующим образом. Будем продолжать использовать знание о том, что среда такси детерминированная, а значит будем продолжать хранить детерминированную матрицу переходов: $(s, a) \rightarrow s'$. Но если раньше мы также хранили **бинарную** маску "выученности" перехода для пар (s,a): $(s, a) \rightarrow 0 | 1$, то теперь будем хранить **степень** выученности $\in [0, 1]$, где 0 означает полное незнание, а 1 - полное знание. Таким образом каждый шаг обучения модели будем повышать это значение, устремляя его к 1.

Будем интерпретировать степень выученности перехода как уверенность в предсказании и использовать как вероятность дать правильное предсказание перехода данной моделью $P[s_{t+1} |s_t,a_t]$. Тогда в случае планирования будущего моделью, она будет с этой вероятностью давать правильный ответ, а с вероятностью $1 - P[s_{t+1} |s_t,a_t]$ будет давать случайное предсказание.

*NB: можно было бы использовать в качестве аппроксиматора нейронную сеть, но мы сознательно останемся в рамках табличных моделей — именно обучение аппроксиматора и будем сейчас эмулировать, но таким образом, чтобы нам было проще контролировать скорость обучения каждого перехода.*

(2 балла)

In [ ]:
class ImperfectModel(Model):
    # обратите внимание, мы наследуемся и переопределяем нужные
    # методы оригинальной идеальной модели

    def __init__(
        self, n_states, n_actions, lr: float, seed: int,
        n_updates: int = 4, buffer_size: int = 1000
    ):
        super().__init__(n_states, n_actions, seed)
        # адаптируйте структуру модели так, чтобы переход (s, a) -> next_s
        # обучался со скоростью `lr`.
        # Для этого поменяем смысл маски посещенных пар (s, a).
        # Заменим бинарную маску на вещественнозначную с числами из [0, 1].
        # Будем интерпретировать их как степень выученности переходов
        # или нашей уверенности в их предсказании
        self.lr = lr
        self.mask_state_action = np.zeros((n_states, n_actions), dtype=float)
        self.buffer = deque(maxlen=buffer_size)
        self.n_updates = n_updates

    def add(self, s: int, a: int, r: float, next_s: int) -> float:
        # Шаг 1: обновим и положим новый опыт в буфер
        new_transition = s, a, r, next_s
        self._update(*new_transition)
        self.buffer.append(new_transition)

        # Шаг 2: сделаем n_updates итераций обучения модели
        # на данных из буфера
        sz_buffer = len(self.buffer)
        for _ in range(min(self.n_updates, sz_buffer)):
            ix = self._rng.integers(sz_buffer)
            self._update(*self.buffer[ix])
        # вернем входное вознаграждение без изменений
        return r

    def _update(self, s: int, a: int, r: float, next_s: int):
        # сохраняем уверенность в предсказании
        confidence = self.mask_state_action[s][a]
        # воспользуемся шагом обучения идеальной модели
        # NB: она перезатрет self.mask_state_action[s][a]
        super().add(s, a, r, next_s)

        # Сделаем шаг обучения уверенности в предсказании
        # перехода со скоростью `lr`.
        # Для простоты, функцию вознаграждения обучаем мгновенно
        # В идеальной модели было:
        # self.mask_state_action[s][a] = 1
        # Теперь будет:
        self.mask_state_action[s][a] = confidence + self.lr * (1 - confidence)

    def predict_transition(self, s, a) -> tuple[float, int, float]:
        # Реализуйте неидеальное предсказание модели перехода:
        # Используйте величину уверенности как вероятность правильно
        # предсказать переход. Иначе будем выдавать случайное предсказание
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        return self.r[s, a], next_s, confidence

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=partial(ImperfectModel, lr=0.02, buffer_size=10000, n_updates=5)
)

log_dyna_q_imp = train(
    env, agent, n_episodes=4000, on_model_updates=5, seed=seed
)

**Вопрос на подумать**: Что можно сказать о результатах модельного обучения в случае, когда модель неидеальна?

## Dyna-Q с неидеальной моделью и внутренней мотивацией на основе ошибки модели

Сигнал ошибки модели является хорошим сигналом мотивации к исследованию, "отправляя" агента в недоисследованные зоны пространства состояний. Как следствие два плюса: а) более быстрое и целенаправленное обучение самой модели среды (которая дает бонус к скорости обучения через обучение в воображении) и б) более целенаправленное исследование самой среды, которое поможет агенту быстрее найти состояния с высоким внешним вознаграждением.

Давайте посмотрим, так ли это. Реализуйте аналогичную предыдущему пункту "неидеальную" внутреннюю модель среды агента, но теперь также добавьте в нее внутреннюю мотивацию. Для этого можно, например, добавлять ошибку предсказания на основе вероятности дать случайное предсказание $1 - P[s_{t+1} |s_t,a_t]$ к полученному внешнему вознаграждению $r_{t+1}$ при обучении функции вознаграждения.

In [ ]:
class IntrinsicMotivationModel(ImperfectModel):
    def __init__(
        self, n_states, n_actions, lr: float, seed: int,
        n_updates: int = 4, buffer_size: int = 1000,
        add_intrinsic: float=0.
    ):
        super().__init__(
            n_states, n_actions, lr, seed,
            n_updates=n_updates, buffer_size=buffer_size
        )
        self.add_intrinsic = add_intrinsic

    def add(self, s: int, a: int, r: float, next_s: int) -> float:
        super().add(s, a, r, next_s)

        # вернем вознаграждение реального перехода
        # с добавкой внутренней мотивации
        r += self._get_intrinsic_reward(s, a)
        return r

    def sample(self) -> tuple[int, int, float, int, float]:
        s, a, r, next_s, confidence = super().sample()

        # вернем вознаграждение предсказанного перехода
        # с добавкой внутренней мотивации
        r += self._get_intrinsic_reward(s, a)
        return s, a, r, next_s, confidence

    def _get_intrinsic_reward(self, s, a):
        # вычислите значение сигнала внутренней мотивации r_im
        # можете адаптировать код ниже.
        r_im = 0.0
        confidence = self.mask_state_action[s][a]
        if self.add_intrinsic > 0.:
            r_im = 1.0 - confidence**3
        return self.add_intrinsic * r_im

Сначала проверим, что все запускается и работает: масштабирующий коэффициент добавочной внутренней мотивации `add_intrinsic` равен 0 $\Rightarrow$ это обычное модельное обучение с неидеальной моделью как в подпункте выше, просто теперь мы использовали объект модели нового класса. Графики должны совпадать

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.02, buffer_size=10000, n_updates=5,
        add_intrinsic=0.
    )
)

log_dyna_q_imp_im = train(
    env, agent, n_episodes=4000, on_model_updates=5, seed=seed
)

In [ ]:
%%time
# а теперь со включенной внутренней мотивацией (см add_intrinsic параметр)
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.025, buffer_size=10000, n_updates=5,
        add_intrinsic=0.1
    )
)

log_dyna_q_imp_im = train(
    env, agent, n_episodes=4000, on_model_updates=5, seed=seed
)

## With imaginary trajectories
До этого процесс обучения по модельным данным происходил беспорядочно — агент случайно сэмплировал состояние, которое ему знакомо, выбирал случайно действие, которое однажды уже выбирал, предсказывал переход (следующее состояние и вознаграждение) и обучался на одном переходе. Процесс повторялся в течение нескольких независимых шагов.

Теперь давайте попробуем воспроизвести процесс обучения в воображении, похожий на планирование из текущей позиции — чтобы агент воображал целую траекторию, начинающуюся с текущего состояния (то есть додумывал/планировал возможное продолжение событий, начиная с текущего момента).

(2 балла)

In [ ]:
class DynaQDreamingAgent(DynaQAgent):
    def dream(self, max_steps, state):
        # текущее состояние — стартовое для траектории длины max_steps в воображении
        s = state
        for _ in range(max_steps):
            # добавьте последовательный шаг в воображении с шагом обучения
            # Действие теперь будем выбирать на основе
            # исследовательской стратегии агента
            # Будем прерывать генерацию траектории на основе
            # уверенности перехода с вероятностью `1 - confidence`
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################
            s = next_s

In [ ]:
%%time
# сначала без внутренней мотивации
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.025, buffer_size=10000, n_updates=5,
        add_intrinsic=0.
    )
)

log_dyna_q_imp_dreamer = train(
    env, agent, n_episodes=4000, on_model_updates=5, seed=seed
)

In [ ]:
%%time
# и то же самое с внутренней мотивацией
env = gym.make("Taxi-v3", max_episode_steps=100)
seed = 1337
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.1,
    gamma=0.975, temp=0.25, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.025, buffer_size=10000, n_updates=5,
        add_intrinsic=0.1
    )
)

log_dyna_q_imp_im_dreamer = train(
    env, agent, n_episodes=4000, on_model_updates=4, seed=seed
)

## Сравнение подходов
Теперь просимулируем задачу посложнее и сравним результаты. В качестве усложнения выбраны:

- более жесткое ограничение на длину эпизода max_episode_steps = 50
- меньшая скорость обучения lr Q-функции и модели
- более жадная стратегия исследования (ниже температура софтмакса)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=Model
)

log_q = train(
    env, agent, n_episodes=15000, on_model_updates=0, seed=seed,
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=Model
)
log_dyna_q = train(env, agent, n_episodes=8000, on_model_updates=5, seed=seed)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(ImperfectModel, lr=0.004, buffer_size=10000, n_updates=5)
)

log_dyna_q_imp = train(
    env, agent, n_episodes=15000, on_model_updates=5, seed=seed
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5,
        add_intrinsic=0.1
    )
)

log_dyna_q_imp_im = train(
    env, agent, n_episodes=10000, on_model_updates=5, seed=seed
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5, add_intrinsic=0.
    )
)

log_dyna_q_imp_dreaming = train(
    env, agent, n_episodes=8000, on_model_updates=5, seed=seed
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5, add_intrinsic=0.1
    )
)

log_dyna_q_imp_im_dreaming = train(
    env, agent, n_episodes=8000, on_model_updates=5, seed=seed
)

In [ ]:
compare_logs([
    (log_q, 'Q-learning'),
    (log_dyna_q, 'Dyna-Q'),
    (log_dyna_q_imp, 'Dyna-Q imperfect model'),
    (log_dyna_q_imp_im, 'Dyna-Q imperfect model + IM'),
    (log_dyna_q_imp_dreaming, 'Dyna-Q imperfect model + Dreaming'),
    (log_dyna_q_imp_im_dreaming, 'Dyna-Q imperfect model + IM + Dreaming'),
])

## Model learning progress

Теперь давайте запустим те же самые эксперименты, но отрисуем средний прогресс обучения переходов в модели (0 — модель полностью необучена, 1 — модель полностью обучила все возможные переходы) и посмотрим, насколько велика разница в скорости исследования среды и обучения модели.

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(ImperfectModel, lr=0.004)
)

_log_dyna_q_imp = train(
    env, agent, n_episodes=5000, on_model_updates=5, seed=seed,
    show_model_progress=True
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.975, eps=0.02, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5,
        add_intrinsic=0.1
    )
)

_log_dyna_q_imp_im = train(
    env, agent, n_episodes=5000, on_model_updates=5, seed=seed,
    show_model_progress=True
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5,
        add_intrinsic=0.
    )
)

_log_dyna_q_imp_dreaming = train(
    env, agent, n_episodes=5000, on_model_updates=5, seed=seed,
    show_model_progress=True
)

In [ ]:
%%time
env = gym.make("Taxi-v3", max_episode_steps=50)
seed = 42
agent = DynaQDreamingAgent(
    env.observation_space.n, env.action_space.n, lr=0.03,
    gamma=0.98, temp=0.1, seed=seed,
    f_model=partial(
        IntrinsicMotivationModel, lr=0.004, buffer_size=10000, n_updates=5,
        add_intrinsic=0.1
    )
)

_log_dyna_q_imp_im_dreaming = train(
    env, agent, n_episodes=5000, on_model_updates=5, seed=seed,
    show_model_progress=True
)

In [ ]:
compare_logs([
    (_log_dyna_q_imp, 'Dyna-Q imperfect model'),
    (_log_dyna_q_imp_im, 'Dyna-Q imperfect model + IM'),
    (_log_dyna_q_imp_dreaming, 'Dyna-Q imperfect model + Dreaming'),
    (_log_dyna_q_imp_im_dreaming, 'Dyna-Q imperfect model + IM + Dreaming'),
])